# Generate figures for publication


In [ ]:
from neuroprocessing.imagingtrials import ImagingTrialLoader
import matplotlib.pyplot as plt
import numpy as np
params = {
        "downsample_factor": 8,
        "aligner_target_num_features": 700,
        "secs_before_stim": 60, # only process frames starting at X seconds before stimulus
        "preprocess_prefix": "aligned_downsampled_",
        "process_prefix": 'processed_',
        "s3fs_toplvl_path": "/Users/ilya_arcadia/arcadia-neuroimaging-pruritogens/Videos",
        "local_toplvl_path": "/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections/",
        "load_from_s3": False,
        "save_to_s3": False,
        'crop_px' : 20,
        'bottom_percentile' : 5
        }

trials = ImagingTrialLoader(params)
exp_dates, trial_names = trials.filter_exp_and_trial_dirs(exp_dir="2024-03-20-F1Num8REP",
                                                          limb='RHL'
                                                        )
# exp_dates, trial_names = trials.filter_exp_and_trial_dirs()

masks = trials.load_mask_files()
traces = trials.load_traces()
sync_infos = trials.get_sync_infos()
# Plot the traces

fig, ax = plt.subplots(figsize=(5,3))
for trace, exp_date, trial_name, sync in zip(traces, exp_dates, trial_names, sync_infos):
    t = (np.arange(0, len(trace))) / (sync['framerate_hz'] / params['downsample_factor']) - params['secs_before_stim']
    trace = trace - trace[np.where(t >= 0)[0][0]]
    # remove slope
    trace = trace - np.polyval(np.polyfit(t, trace, 1), t)
    ax.plot(t, trace, label=exp_date + ' ' + trial_name)
# legend to the right of the plot
# ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
ax.axvline(x=0, color='r', linestyle='--')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Response (a.u.)')
ax.set_xlim([-50, 600])
# plot masks
fig, axs = plt.subplots(ncols=len(masks), figsize=(15, 7))
for ax, mask, exp_date, trial_name in zip(axs, masks, exp_dates, trial_names):
    ax.imshow(mask, cmap='gray')
    # remove axis ticks
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(exp_date + ' ' + trial_name, fontsize=8)
plt.tight_layout()
